In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time 
import pandas as pd
import pymongo
from pymongo import MongoClient
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Document, Text, Integer
from elasticsearch.helpers import bulk
import numpy as np

In [3]:
df = pd.read_csv('data_youtube.csv',delimiter=',')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1477 entries, 0 to 1476
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Jeu                       1477 non-null   object
 1   Titre                     1477 non-null   object
 2   Chaîne                    1477 non-null   object
 3   Nombre de vues            1477 non-null   object
 4   Date de la mise en ligne  1477 non-null   object
dtypes: object(5)
memory usage: 57.8+ KB


In [4]:
df2 = pd.concat([df['Titre'],df['Nombre de vues']],axis=1)
df2 = df2.drop_duplicates()
df2

,Titre,Nombre de vues
0,Trevor Didn't Forget 💀 || GTA V,"1,8 M de vues"
1,Completing the most painful dares in GTA 5,772 k vues
2,Which car can climb to the highest level in GT...,1 M de vues
3,Robbing a MOPAR DEALERSHIP in GTA 5 RP!,434 k vues
4,SHORTEST vs TALLEST PERSON in GTA 5!,654 k vues
...,...,...
1472,DragonFlight Gameplay. Unboxing Unclaimed Blac...,129 vues
1473,Allison's World of Warcraft Stream,53 vues
1474,I wish I found a girl in World Of Warcraft,339 vues
1475,Level With Me (Part 4) || World of Warcraft,41 vues


In [11]:
data = df.to_dict('records')

In [12]:
data

[{'Jeu': 'Grand Theft Auto V',
  'Titre': "Trevor Didn't Forget 💀 || GTA V",
  'Chaîne': 'HigHub',
  'Nombre de vues': '1,8 M de vues',
  'Date de la mise en ligne': 'il y a 5 jours'},
 {'Jeu': 'Grand Theft Auto V',
  'Titre': 'Completing the most painful dares in GTA 5',
  'Chaîne': 'GrayStillPlays',
  'Nombre de vues': '772 k vues',
  'Date de la mise en ligne': 'il y a 6 jours'},
 {'Jeu': 'Grand Theft Auto V',
  'Titre': 'Which car can climb to the highest level in GTA 5?',
  'Chaîne': 'GrayStillPlays',
  'Nombre de vues': '1 M de vues',
  'Date de la mise en ligne': 'il y a 2 jours'},
 {'Jeu': 'Grand Theft Auto V',
  'Titre': 'Robbing a MOPAR DEALERSHIP in GTA 5 RP!',
  'Chaîne': 'FuriousFade',
  'Nombre de vues': '434 k vues',
  'Date de la mise en ligne': 'il y a 6 jours'},
 {'Jeu': 'Grand Theft Auto V',
  'Titre': 'SHORTEST vs TALLEST PERSON in GTA 5!',
  'Chaîne': 'Caylus',
  'Nombre de vues': '654 k vues',
  'Date de la mise en ligne': 'il y a 4 jours'},
 {'Jeu': 'Grand Theft 

In [13]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200,"scheme": "http"}])

In [14]:
es.ping()

True

In [16]:
def generate_data(data):
    for docu in data:
        yield {
            "_index": "yt_twitch",
            "_type": "video",
            "_source": {k:v if v else None for k,v in docu.items()},
        }
        
if es.indices.exists('yt_twitch')==True:
    es.indices.delete(index='yt_twitch')
    bulk(es, generate_data(data))
else :
    bulk(es, generate_data(data))

In [26]:
QUERY={"query": {
    "bool": {
      "must": [],
      "filter": [
        {
          "bool": {
            "should": [
              {
                "match": {
                  "Jeu.keyword": "League of Legends"
                }
              }
            ],
            "minimum_should_match": 1
          }
        }
      ],
      "should": [],
      "must_not": []
    }
  },}

  

In [27]:
result = es.search(index="yt_twitch", body=QUERY, size = 1500)

In [28]:
[elt['_source'] for elt in result["hits"]["hits"]]

[{'Jeu': 'League of Legends',
  'Titre': 'The Mageseeker: A League of Legends Story | Official Teaser Trailer',
  'Chaîne': 'Riot Forge',
  'Nombre de vues': '585 k vues',
  'Date de la mise en ligne': 'il y a 3 jours'},
 {'Jeu': 'League of Legends',
  'Titre': 'Riot Revealed Their Plan To SAVE League!',
  'Chaîne': 'Necrit',
  'Nombre de vues': '289 k vues',
  'Date de la mise en ligne': 'il y a 7 jours'},
 {'Jeu': 'League of Legends',
  'Titre': 'These 2 are BREAKING League of Legends together...',
  'Chaîne': 'Vandiril',
  'Nombre de vues': '336 k vues',
  'Date de la mise en ligne': 'il y a 4 jours'},
 {'Jeu': 'League of Legends',
  'Titre': 'Milio Release Date - League of Legends New Champion',
  'Chaîne': 'Ryscu',
  'Nombre de vues': '141 k vues',
  'Date de la mise en ligne': 'il y a 4 jours'},
 {'Jeu': 'League of Legends',
  'Titre': 'NEW TIER LIST for PATCH 13.4: HUGE Changes! - SEASON 13',
  'Chaîne': 'Skill Capped Challenger LoL Guides',
  'Nombre de vues': '147 k vues',
  '

In [30]:
result = es.search(index="yt_twitch", body={"query": {"match_all": {}}},size = 1500)
result

{'took': 31,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1477, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'yt_twitch',
    '_type': 'video',
    '_id': 'r4n1iIYBTm0hOOWjddgo',
    '_score': 1.0,
    '_source': {'Jeu': 'Grand Theft Auto V',
     'Titre': "Trevor Didn't Forget 💀 || GTA V",
     'Chaîne': 'HigHub',
     'Nombre de vues': '1,8 M de vues',
     'Date de la mise en ligne': 'il y a 5 jours'}},
   {'_index': 'yt_twitch',
    '_type': 'video',
    '_id': 'sIn1iIYBTm0hOOWjddgo',
    '_score': 1.0,
    '_source': {'Jeu': 'Grand Theft Auto V',
     'Titre': 'Completing the most painful dares in GTA 5',
     'Chaîne': 'GrayStillPlays',
     'Nombre de vues': '772 k vues',
     'Date de la mise en ligne': 'il y a 6 jours'}},
   {'_index': 'yt_twitch',
    '_type': 'video',
    '_id': 'sYn1iIYBTm0hOOWjddgo',
    '_score': 1.0,
    '_source': {'Jeu': 'Grand Theft Auto V',
     'Titre': 'Which